In [17]:
#1. Create a local database of your personal film profile by scraping letterboxd
#1a. We're going to implement a tree search to do this. Starting from my user_id, and branching out using a defined formula.
#1b. For each user_id, we will find a list of films from their diary.
#1c. For each film that they rated 5 stars, we'll add that to our 'good_films_set'
#1d. For each film in the 'good_films_list', we'll find a list of users who rated it 5 stars, and add them to our 'users_set'
#1e. Then, iterate the search.
#1f. We're planning on building a 'total_space_of_films_users_etc.', but focussing on 'good_users' first in case our IP address gets blocked.

#2. Create a Recommender Engine that takes my letterbox ratings and finds the users with the closest ratings
#2a.

#3. Create a simple interface (TBD)

In [117]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib import request
import re
import pickle



In [78]:
def scrape_user(user_id):
    # returns one of [error, FilmObject[] ]
    page = 1 # TODO - ITERATE THROUGH PAGES
    url = f'https://letterboxd.com/{user_id}/films/diary/page/{page}' 
    req = request.Request(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'})
    c = request.urlopen(req)
    r = c.read().decode('utf-8')
    soup = BeautifulSoup(r,'html.parser')
    
    return soup

def scrape_film(film_id):
    # returns one of [error, UserObject[] ]
    rating = 5
    page = 1 # TODO - ITERATE THROUGH PAGES
    url = f'https://letterboxd.com/film/{film_id}/members/rated/{rating}/page/{page}'
    req = request.Request(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'})
    c = request.urlopen(req)
    r = c.read().decode('utf-8')
    soup = BeautifulSoup(r,'html.parser')
    
    return soup

In [41]:
def get_user_diary(user_id):
    page = 1
    while True:
        try:
            data = scrape_user(user_id)
            page += 1
            sleep(1)
        except:
            break

In [42]:
def get_film_raters(film_id, rating):
    page = 1
    while True:
        try:
            rating = 10
            data = scrape_film(film_id)
            page += 1
            sleep(1)
        except:
            break

In [43]:
'''

Users: user_id[]

FilmObject: {
    film_id: string,
    rating: int<1-10>,
    ...
}

data = {
    user_id: {
        films: FilmObject[],
    }
}

'''

'\n\nUsers: user_id[]\n\nFilmObject: {\n    film_id: string,\n    rating: int<1-10>,\n    ...\n}\n\ndata = {\n    user_id: {\n        films: FilmObject[],\n    }\n}\n\n'

In [116]:
user_id = 'ncklake'

soup = scrape_user(user_id)
films = soup.find_all('tr', class_='diary-entry-row')

pattern = r'/film/(.*?)/'

for film in films:
    # Find the film-slug using a pattern in class name
    film_details = film.find('div', class_=re.compile('^really-lazy-load poster film-poster film-poster-\d+ linked-film-poster$'))
    film_slug = film_details['data-film-slug']
    
    match = re.search(pattern, film_slug)
    film_id = match.group(1) if match else None
    
    print(film_id)

    # Find the rating using a pattern in class name
    rating_details = film.find('input', class_=re.compile('^rateit-field diary-rating-\d+$'))
    rating = int(rating_details['value'])
    print(rating)

oppenheimer-2023
10
moneyball
9
chronicle
5
shutter-island
10
role-models
6
jerry-maguire
9
spider-man-across-the-spider-verse
9
theres-no-place-like-home
8
el-camino-a-breaking-bad-movie
8
white-chicks
8
ant-man-and-the-wasp-quantumania
5
boyz-n-the-hood
9
hustle-2022
9
warrior
10
stand-by-me
3
abraham-lincoln-vampire-hunter
3
the-departed
10
bullet-train
9
the-martian
8
shrek
8
monty-python-and-the-holy-grail
10
friday
9
speed
7
up
8
the-grand-budapest-hotel
8
catch-me-if-you-can-2002
7
ready-player-one
8
uncut-gems
8
walle
9
shaun-of-the-dead
8
guardians-of-the-galaxy
9
john-wick
8
spider-man-no-way-home
8
rio-2011
7
the-wolf-of-wall-street
7
the-intouchables
9
casablanca
9
kong-skull-island
5
avatar
5
shazam
7
the-batman
9
donnie-darko
7
birdemic-shock-and-terror
0
the-suicide-squad
6
the-social-network
9
the-good-the-bad-and-the-ugly
10
fantastic-mr-fox
8
back-to-the-future
5
no-country-for-old-men
9
the-usual-suspects
10


In [59]:
films[1].find('div', class_=re.compile('^really-lazy-load poster film-poster film-poster-\d+ linked-film-poster$'))

<div class="really-lazy-load poster film-poster film-poster-6286 linked-film-poster" data-cache-busting-key="0880b9ae" data-film-id="6286" data-film-slug="/film/moneyball/" data-hide-tooltip="true" data-image-height="52" data-image-width="35" data-linked="linked" data-poster-url="/film/moneyball/image-150/" data-target-link="/ncklake/film/moneyball/" data-target-link-target=""> <img alt="Moneyball" class="image" height="52" src="https://s2.ltrbxd.com/static/img/empty-poster-35.9df54d4d.png" width="35"/> <span class="frame"><span class="frame-title"></span></span> </div>

In [114]:
film_id = 'oppenheimer-2023'

soup = scrape_film(film_id)

pattern = r'/([^/]*)/'

user_table = soup.find('table', class_='person-table film-table').find('tbody').find_all('tr')

users = []

for user in user_table:

    user_id = user.find('td').find('a')['href']

    match = re.search(pattern, user_id)
    user_id = match.group(1) if match else None
    
    users.append(user_id)


carterk2
ampre
daltonisafraid
laraelias
carmainz
ramikfd
brendan_clotz
captaintony
ashtenwickham
maiquell
lorene21
ranvirbhaya
brunolcs
floptrs
ezio_altair
merecatta
badhampreschool
katiespeedd
daysofheaven
joshlim
amen80
stijntr
scenesofnoir
lassebask
angussheldon


In [118]:
# Open a file for writing
with open("user_table-10.pkl", "wb") as f:
    # Pickle the data dictionary using the highest protocol available
    pickle.dump(user_table, f, pickle.HIGHEST_PROTOCOL)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [119]:
with open("user_table-10.pkl", "rb") as f:
    # Load the pickled data
    data = pickle.load(f)

print(data)

EOFError: Ran out of input